In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"
#r "nuget: Microsoft.ML, 1.5.5"
#r "nuget: Microsoft.ML.Mkl.Components, 1.5.5"
#r "nuget: Microsoft.ML.TimeSeries, 1.5.5"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha5"

In [ ]:
open Deedle
open Plotly.NET

In [ ]:
let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

In [ ]:
let shiftedValues =
    data?Value
    |> Series.shift -1

let baseDateSeries =
    Seq.zip data.RowKeys data.RowKeys
    |> Series.ofObservations

let dayOfWeek =
    baseDateSeries
    |> Series.mapValues (fun dt -> string dt.DayOfWeek)

let month =
    baseDateSeries
    |> Series.mapValues (fun dt -> string dt.Month)

let peakOffPeak =
    baseDateSeries
    |> Series.mapValues (fun dt -> if dt.Hour < 8 || dt.Hour > 19 then "OffPeak" else "Peak")

let dataWithFeatures =
    data
    |> Frame.addCol "Target" shiftedValues
    |> Frame.addCol "DayOfWeek" dayOfWeek
    |> Frame.addCol "Month" month
    |> Frame.addCol "PeakOffPeak" peakOffPeak
    |> Frame.filterRows (fun key _ -> key.Year < 2020)
    |> Frame.dropSparseRows

let dataTrain =
    dataWithFeatures
    |> Frame.filterRows (fun key _ -> key.Year < 2019)

let dataTest =
    dataWithFeatures
    |> Frame.filterRows (fun key _ -> key.Year >= 2019)

dataTrain
|> Frame.skip 3

In [ ]:
dataTrain |> Frame.takeLast 1

In [ ]:
type ILoadRow =
    abstract member Ticks: float32 with get
    abstract member Value: float32 with get
    abstract member Target: float32 with get
    abstract member DayOfWeek: string with get
    abstract member Month: string with get
    abstract member PeakOffPeak: string with get

let (trainKeys: DateTime seq, trainRows: ILoadRow seq) =
    dataTrain.GetRowsAs<ILoadRow>()
    |> Series.observations
    |> Seq.unzip

let testKeys, testRows =
    dataTest.GetRowsAs<ILoadRow>()
    |> Series.observations
    |> Seq.unzip

Seq.length trainRows, Seq.length testRows

In [ ]:
open Microsoft.ML
open Microsoft.ML.Data
open Microsoft.ML.Trainers
open Microsoft.ML.Transforms
open FSharp.Stats.Correlation

In [ ]:
[<CLIMutable>]
type ForecastInput =
    { Ticks: float32
      Value: float32
      [<ColumnName("Label")>]Target: float32
      DayOfWeek: string
      Month: string
      PeakOffPeak: string }

    static member FromILoadRows (row: ILoadRow) =
        { Ticks = row.Ticks
          Value = row.Value
          Target = row.Target
          DayOfWeek = row.DayOfWeek
          Month = row.Month
          PeakOffPeak = row.PeakOffPeak }

[<CLIMutable>]
type ForecastResult =
    { [<ColumnName("Score")>]LoadForecast: float32 }

let downCastPipeline (pipeline: IEstimator<'a>) =
    match pipeline with
    | :? IEstimator<ITransformer> as p -> p
    | _ -> failwith $"The pipeline has to be an instance of IEstimator<ITransformer> but was %A{pipeline.GetType()}"

let mlContext = MLContext(seed = 42)
let defInp = Unchecked.defaultof<ForecastInput>
let dayOneHot = "DayOfWeekOneHot"
let monthOneHot = "MonthOneHot"
let peakOneHot = "PeakOffPeakOneHot"

let processingPipeline =
    EstimatorChain()
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(dayOneHot, nameof defInp.DayOfWeek))
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(monthOneHot, nameof defInp.Month))
        .Append(mlContext.Transforms.Categorical.OneHotEncoding(peakOneHot, nameof defInp.PeakOffPeak))
        .Append(mlContext.Transforms.Concatenate("Features", [| dayOneHot; monthOneHot; peakOneHot; nameof defInp.Ticks |]))
        // .Append(mlContext.Transforms.Concatenate("Features", [| nameof defInp.Value |]))
        // .Append(mlContext.Transforms.Concatenate("Features", [| dayOneHot; monthOneHot; peakOneHot; nameof defInp.Ticks; nameof defInp.Value |]))
    |> downCastPipeline

let trainerOptions = OlsTrainer.Options(CalculateStatistics = true)
let trainer =
    mlContext.Regression.Trainers.Ols(trainerOptions)
    |> downCastPipeline

let dataViewTrain = mlContext.Data.LoadFromEnumerable<ForecastInput>(trainRows |> Seq.map (fun row -> ForecastInput.FromILoadRows row))
let dataViewTest = mlContext.Data.LoadFromEnumerable<ForecastInput>(testRows |> Seq.map (fun row -> ForecastInput.FromILoadRows row))

In [ ]:
let trainingPipeline = processingPipeline.Append(trainer)
let trainedModel = trainingPipeline.Fit(dataViewTrain)

In [ ]:
let transformedData = trainedModel.Transform(dataViewTest)
let predictions = mlContext.Data.CreateEnumerable<ForecastResult>(transformedData, reuseRowObject = false) |> Seq.toList

mlContext.Regression.Evaluate(transformedData)

In [ ]:
open Plotly.NET

let predVals = predictions |> Seq.map (fun p -> p.LoadForecast)
let actualVals = testRows |> Seq.map (fun r -> r.Target)

let predChart =
    Seq.zip testKeys predVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip testKeys actualVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

PValues are unfortunately unreliable. See [this issue](https://github.com/dotnet/machinelearning/issues/5696) for more context.

In [ ]:
let model = (trainedModel.LastTransformer :?> RegressionPredictionTransformer<Microsoft.ML.Trainers.OlsModelParameters>).Model
model

In [ ]:
let minVal =
    min (Seq.min predVals) (Seq.min actualVals)
    |> float
    |> fun v -> v - 100.

let largestVal =
    max (Seq.max predVals) (Seq.max actualVals)
    |> float
    |> fun v -> v + 100.

let diagonalLine =
    [ (minVal, minVal); (largestVal, largestVal) ]
    |> fun xy -> Chart.Line(xy, Name = "Diagonal")

let predActualScatter =
    Seq.zip predVals actualVals
    |> fun xy -> Chart.Point(xy, UseWebGL = true, Name = "Pred/Actual")
    |> Chart.withX_AxisStyle ("predictions", MinMax = (minVal, largestVal))
    |> Chart.withY_AxisStyle ("actual", MinMax = (minVal, largestVal))

[ predActualScatter; diagonalLine ]
|> Chart.Combine
|> display

Seq.pearson actualVals predVals
|> display

In [ ]:
let predEngine = mlContext.Model.CreatePredictionEngine<ForecastInput, ForecastResult>(trainedModel)

let inputs =
    mlContext.Data.CreateEnumerable<ForecastInput>(dataViewTest, reuseRowObject = false)

let firstInput =
    Seq.head inputs
    |> fun fi -> { fi with Target = float32 0. }

let theRest =
    inputs
    |> Seq.skip 1
    |> Seq.map (fun fi -> { fi with Value = float32 0.; Target = float32 0. })
    |> Seq.toList

firstInput::theRest
|> List.take 5

In [ ]:
let rec predictDynamically (toPredict: ForecastInput list) (lastPredcition: ForecastResult) =
    match toPredict with
    | [] -> []
    | x::xs ->
        let x_hat = { x with Value = lastPredcition.LoadForecast }
        let forecasted = predEngine.Predict(x_hat)
        forecasted::(predictDynamically xs forecasted)

let dynamicPredictions = predictDynamically (firstInput::theRest) { LoadForecast = firstInput.Value }
dynamicPredictions
|> List.take 10

In [ ]:
let dynamicPredVals = dynamicPredictions |> List.map (fun pred -> pred.LoadForecast)

let predChart =
    Seq.zip testKeys dynamicPredVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip testKeys actualVals
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

In [ ]:
let modelDirectory = "../models"
let linearModel = modelDirectory + "/linear_model.zip"

mlContext.Model.Save(trainedModel, dataViewTrain.Schema, linearModel)

In [ ]:
[<CLIMutable>]
type AlternativeForecastInput =
    { Load: float32
      TimeStamp: DateTime }

[<CLIMutable>]
type AlternativeLoadForecast =
  { Forecast: float32 array
    LowerBound: float32 array
    UpperBound: float32 array }

let altForecastInputs =
  dataTrain?Value
  |> Series.observations
  |> Seq.map (fun (k, v) -> { Load = float32 v; TimeStamp = k})

let pipeline =
  mlContext.Forecasting.ForecastBySsa(
    "Forecast",
    nameof Unchecked.defaultof<AlternativeForecastInput>.Load,
    windowSize =  24 * 30,
    seriesLength = 24 * 30 * 2,
    trainSize = dataTrain.RowCount,
    horizon = 24 * 30 ,
    confidenceLevel = 0.90f,
    confidenceLowerBoundColumn = "LowerBound",
    confidenceUpperBoundColumn = "UpperBound"
)

let altForecastData = mlContext.Data.LoadFromEnumerable(altForecastInputs)

let model = pipeline.Fit(altForecastData)

In [ ]:
open Microsoft.ML.Transforms.TimeSeries

let forecastingEngine = model.CreateTimeSeriesEngine<AlternativeForecastInput, AlternativeLoadForecast>(mlContext)

let horizon = 24 * 5
let forecast = forecastingEngine.Predict(horizon = horizon)

In [ ]:
let predChart =
    Seq.zip (testKeys |> Seq.take horizon) forecast.Forecast
    |> fun xy -> Chart.Range(xy,
                             forecast.LowerBound,
                             forecast.UpperBound,
                             mode = StyleParam.Mode.Lines,
                             Color = Colors.toWebColor Colors.Table.Office.blue,
                             RangeColor = Colors.toWebColor Colors.Table.Office.lightBlue)
    |> Chart.withTraceName "Forecast_CI"

let actualChart =
    Seq.zip (testKeys |> Seq.take horizon) (testRows |> Seq.take horizon |> Seq.map (fun r -> r.Target))
    |> fun xy -> Chart.Line(xy,
                            Color = Colors.toWebColor Colors.Table.Office.orange,
                            UseWebGL = true,
                            Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

In [ ]:
let actualVals = testRows |> Seq.take (24 * 30) |> Seq.map (fun r -> r.Target)
let predVals = forecast.Forecast

let minVal =
    min (Seq.min predVals) (Seq.min actualVals)
    |> float
    |> fun v -> v - 100.

let largestVal =
    max (Seq.max predVals) (Seq.max actualVals)
    |> float
    |> fun v -> v + 100.

let diagonalLine =
    [ (minVal, minVal); (largestVal, largestVal) ]
    |> fun xy -> Chart.Line(xy, Name = "Diagonal")

let predActualScatter =
    Seq.zip predVals actualVals
    |> fun xy -> Chart.Point(xy, UseWebGL = true, Name = "Pred/Actual")
    |> Chart.withX_AxisStyle ("predictions", MinMax = (minVal, largestVal))
    |> Chart.withY_AxisStyle ("actual", MinMax = (minVal, largestVal))

[ predActualScatter; diagonalLine ]
|> Chart.Combine
|> display

Seq.pearson actualVals predVals
|> display

In [ ]:
let forecastModel = modelDirectory + "/forecast_model.zip"

mlContext.Model.Save(model, altForecastData.Schema, forecastModel)